<a href="https://colab.research.google.com/github/zolfaShefreie/spark_basics/blob/main/pyspark_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##install and import pyspark library

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 31 kB/s 
     |████████████████████████████████| 198 kB 59.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=45fbe529c06761565f02063324489c48b18dcf1f1f53fe2493cf20c315654f68
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('train').getOrCreate()

In [4]:
sparkContent = spark.sparkContext

In [5]:
import requests

def download_file(url, file_name):
  file_content = requests.get(url).text
  file = open(file_name, 'w')
  file.write(file_content)
  file.close()

##Load Book names

In [6]:
import pandas as pd

url = "https://gist.githubusercontent.com/jaidevd/23aef12e9bf56c618c41/raw/c05e98672b8d52fa0cb94aad80f75eb78342e5d4/books.csv"
file_name = "books.csv"

In [7]:
download_file(url, file_name)

In [8]:
book_names = pd.read_csv(file_name)['Title'].head(50).to_list()
book_names

['Fundamentals of Wavelets',
 'Data Smart',
 'God Created the Integers',
 'Superfreakonomics',
 'Orientalism',
 'Nature of Statistical Learning Theory, The',
 'Integration of the Indian States',
 "Drunkard's Walk, The",
 'Image Processing & Mathematical Morphology',
 'How to Think Like Sherlock Holmes',
 'Data Scientists at Work',
 'Slaughterhouse Five',
 'Birth of a Theorem',
 'Structure & Interpretation of Computer Programs',
 'Age of Wrath, The',
 'Trial, The',
 "Statistical Decision Theory'",
 'Data Mining Handbook',
 'New Machiavelli, The',
 'Physics & Philosophy',
 'Making Software',
 'Analysis, Vol I',
 'Machine Learning for Hackers',
 'Signal and the Noise, The',
 'Python for Data Analysis',
 'Introduction to Algorithms',
 'Beautiful and the Damned, The',
 'Outsider, The',
 'Complete Sherlock Holmes, The - Vol I',
 'Complete Sherlock Holmes, The - Vol II',
 'Wealth of Nations, The',
 'Pillars of the Earth, The',
 'Mein Kampf',
 'Tao of Physics, The',
 "Surely You're Joking Mr F

In [9]:
books_rdd = sparkContent.parallelize(book_names)

In [ ]:
books_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [10]:
books_rdd.zipWithIndex().filter(lambda x: x[1]==19).map(lambda x: x[0]).collect()

['Physics & Philosophy']

In [11]:
books_rdd.map(lambda x: x.upper()).collect()

['FUNDAMENTALS OF WAVELETS',
 'DATA SMART',
 'GOD CREATED THE INTEGERS',
 'SUPERFREAKONOMICS',
 'ORIENTALISM',
 'NATURE OF STATISTICAL LEARNING THEORY, THE',
 'INTEGRATION OF THE INDIAN STATES',
 "DRUNKARD'S WALK, THE",
 'IMAGE PROCESSING & MATHEMATICAL MORPHOLOGY',
 'HOW TO THINK LIKE SHERLOCK HOLMES',
 'DATA SCIENTISTS AT WORK',
 'SLAUGHTERHOUSE FIVE',
 'BIRTH OF A THEOREM',
 'STRUCTURE & INTERPRETATION OF COMPUTER PROGRAMS',
 'AGE OF WRATH, THE',
 'TRIAL, THE',
 "STATISTICAL DECISION THEORY'",
 'DATA MINING HANDBOOK',
 'NEW MACHIAVELLI, THE',
 'PHYSICS & PHILOSOPHY',
 'MAKING SOFTWARE',
 'ANALYSIS, VOL I',
 'MACHINE LEARNING FOR HACKERS',
 'SIGNAL AND THE NOISE, THE',
 'PYTHON FOR DATA ANALYSIS',
 'INTRODUCTION TO ALGORITHMS',
 'BEAUTIFUL AND THE DAMNED, THE',
 'OUTSIDER, THE',
 'COMPLETE SHERLOCK HOLMES, THE - VOL I',
 'COMPLETE SHERLOCK HOLMES, THE - VOL II',
 'WEALTH OF NATIONS, THE',
 'PILLARS OF THE EARTH, THE',
 'MEIN KAMPF',
 'TAO OF PHYSICS, THE',
 "SURELY YOU'RE JOKING MR F

In [14]:
books_rdd.map(lambda x: (x[0].upper(), x)).\
groupBy(lambda x:x[0]).\
map(lambda group: (group[0], [x[1] for x in group[1]])).\
sortBy(lambda x: x[0]).collect()

[('A', ['Age of Wrath, The', 'Analysis, Vol I', 'Asami Asami']),
 ('B', ['Birth of a Theorem', 'Beautiful and the Damned, The']),
 ('C',
  ['Complete Sherlock Holmes, The - Vol I',
   'Complete Sherlock Holmes, The - Vol II']),
 ('D',
  ['Data Smart',
   "Drunkard's Walk, The",
   'Data Scientists at Work',
   'Data Mining Handbook']),
 ('F',
  ['Fundamentals of Wavelets',
   'Farewell to Arms, A',
   'False Impressions',
   'Freakonomics']),
 ('G', ['God Created the Integers']),
 ('H', ['How to Think Like Sherlock Holmes', 'Hidden Connections, The']),
 ('I',
  ['Integration of the Indian States',
   'Image Processing & Mathematical Morphology',
   'Introduction to Algorithms']),
 ('J', ['Jurassic Park', 'Journal of a Novel']),
 ('L', ['Last Lecture, The']),
 ('M',
  ['Making Software',
   'Machine Learning for Hackers',
   'Mein Kampf',
   'Moon is Down, The']),
 ('N', ['Nature of Statistical Learning Theory, The', 'New Machiavelli, The']),
 ('O', ['Orientalism', 'Outsider, The', 'Onc

##Load Shakespeare file

In [15]:
txt_url = "https://raw.githubusercontent.com/brunoklein99/deep-learning-notes/master/shakespeare.txt"
txt_file_name = "shakespeare.txt"

In [16]:
download_file(txt_url, txt_file_name)

## preprocessing and map-reduce

In [17]:
def delete_puctuation(x):
  new_str = str() + x
  for each in '''!()-[]{.};:'"\,<>/?@#$%^&*_~`|’“”…—–''':
    new_str = new_str.replace(each, "")
  return new_str.strip()

In [18]:
text_rdd = sparkContent.textFile(txt_file_name)
words = text_rdd.flatMap(lambda line: line.split(" ")).filter(lambda x: x.strip())
words = words.map(delete_puctuation).map(lambda x: x.lower())
words.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b).collect()

[('shakespeare', 1),
 ('fairest', 5),
 ('creatures', 2),
 ('we', 15),
 ('increase', 4),
 ('thereby', 2),
 ('beautys', 18),
 ('rose', 6),
 ('never', 15),
 ('die', 12),
 ('but', 163),
 ('as', 121),
 ('riper', 2),
 ('his', 107),
 ('tender', 7),
 ('heir', 3),
 ('bear', 12),
 ('thou', 235),
 ('thine', 44),
 ('own', 30),
 ('bright', 11),
 ('eyes', 56),
 ('thy', 287),
 ('flame', 3),
 ('selfsubstantial', 1),
 ('fuel', 1),
 ('making', 12),
 ('famine', 1),
 ('where', 40),
 ('abundance', 4),
 ('self', 77),
 ('sweet', 55),
 ('now', 46),
 ('worlds', 5),
 ('ornament', 5),
 ('only', 6),
 ('herald', 1),
 ('churl', 2),
 ('in', 323),
 ('pity', 8),
 ('world', 27),
 ('this', 105),
 ('due', 6),
 ('grave', 3),
 ('when', 106),
 ('forty', 1),
 ('winters', 5),
 ('besiege', 2),
 ('brow', 8),
 ('dig', 1),
 ('field', 1),
 ('youths', 1),
 ('weed', 3),
 ('of', 370),
 ('worth', 19),
 ('beauty', 52),
 ('treasure', 9),
 ('days', 18),
 ('say', 28),
 ('an', 17),
 ('praise', 28),
 ('more', 64),
 ('use', 13),
 ('couldst',